In [5]:
from pipelines.util.platform import start_spark
from pipelines.util.storage import read_csv,read_csvfolder
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql.window import Window
from pyspark import SparkConf
import pandas as pd

In [6]:
spark, logger, conf = start_spark(app_name='colibri-challange-profiledata')

local


In [7]:
dg1 = read_csv(spark, "{storage_landed}/data_group_1.csv".format(storage_landed=spark.conf.get('storage.landed')), dict(header="true",inferSchema="true") )

In [8]:
dg1.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- turbine_id: integer (nullable = true)
 |-- wind_speed: double (nullable = true)
 |-- wind_direction: integer (nullable = true)
 |-- power_output: double (nullable = true)



In [3]:
sch = T.StructType([
    T.StructField("timestamp", T.TimestampType(), True),
    T.StructField("turbine_id", T.IntegerType(), True),
    T.StructField("wind_speed", T.DoubleType(), True),
    T.StructField("wind_direction", T.IntegerType(), True),
    T.StructField("power_output", T.DoubleType(), True)
])
csvoptions = {
    "header": "true",
    "sep": ",",
    "schema": sch,
    "inferSchema": "false"
}
dg_1_5 = read_csvfolder(spark, "{storage_landed}".format(storage_landed=spark.conf.get('storage.landed')), csvoptions). \
    withColumn("filename", F.input_file_name())

In [12]:
#dg_1_5.printSchema()
dg_1_5.count()
#dg_1_5.orderBy("timestamp","turbine_id").show(truncate=False)

11160

In [27]:
df_dcount = dg_1_5.groupBy(F.col("turbine_id"),F.to_date(F.col("timestamp")).alias("tdate")).count()
df_dcount.orderBy(F.col("turbine_id"),F.col("tdate")).show(n=100)


+----------+----------+-----+
|turbine_id|     tdate|count|
+----------+----------+-----+
|         1|2022-03-01|   24|
|         1|2022-03-02|   24|
|         1|2022-03-03|   24|
|         1|2022-03-04|   24|
|         1|2022-03-05|   24|
|         1|2022-03-06|   24|
|         1|2022-03-07|   24|
|         1|2022-03-08|   24|
|         1|2022-03-09|   24|
|         1|2022-03-10|   24|
|         1|2022-03-11|   24|
|         1|2022-03-12|   24|
|         1|2022-03-13|   24|
|         1|2022-03-14|   24|
|         1|2022-03-15|   24|
|         1|2022-03-16|   24|
|         1|2022-03-17|   24|
|         1|2022-03-18|   24|
|         1|2022-03-19|   24|
|         1|2022-03-20|   24|
|         1|2022-03-21|   24|
|         1|2022-03-22|   24|
|         1|2022-03-23|   24|
|         1|2022-03-24|   24|
|         1|2022-03-25|   24|
|         1|2022-03-26|   24|
|         1|2022-03-27|   24|
|         1|2022-03-28|   24|
|         1|2022-03-29|   24|
|         1|2022-03-30|   24|
|         

In [6]:
wspec = Window.partitionBy(F.col("turbine_id"),F.to_date(F.col("timestamp"))).orderBy(F.col("timestamp"))

dg_1_5.withColumn("prev_ts",F.lag(F.col("timestamp"),1).over(wspec)). \
    withColumn("diff_hrs", (F.col("timestamp").cast("long") - F.col("prev_ts").cast("long"))/3600). \
    filter( (F.col("diff_hrs") > 1.0) | (F.col("diff_hrs") < 1.0) ).show(truncate=False)


# df_sum = dg_1_5.groupBy(F.col("turbine_id"),F.to_date(F.col("timestamp")).alias("tdate")).agg(F.sum(F.unix_timestamp(F.col("timestamp"))).alias("tsum"))
# df_sum.orderBy(F.col("turbine_id"),F.col("tdate")).show()


+-------------------+----------+----------+--------------+------------+--------------------------------------------------------------------------+-------------------+--------+
|timestamp          |turbine_id|wind_speed|wind_direction|power_output|filename                                                                  |prev_ts            |diff_hrs|
+-------------------+----------+----------+--------------+------------+--------------------------------------------------------------------------+-------------------+--------+
|2022-03-27 02:00:00|1         |10.9      |175           |1.9         |file:///C:/PalProjects/colibri-challange/code/data/landed/data_group_1.csv|2022-03-27 02:00:00|0.0     |
|2022-03-27 02:00:00|2         |9.5       |94            |4.4         |file:///C:/PalProjects/colibri-challange/code/data/landed/data_group_1.csv|2022-03-27 02:00:00|0.0     |
|2022-03-27 02:00:00|3         |13.4      |110           |3.0         |file:///C:/PalProjects/colibri-challange/code/dat

In [5]:
dg_1_5.filter(F.to_date(F.col("timestamp")) == '2022-03-27').orderBy(F.col("turbine_id"),F.col("timestamp")).show(truncate=False)

+-------------------+----------+----------+--------------+------------+--------------------------------------------------------------------------+
|timestamp          |turbine_id|wind_speed|wind_direction|power_output|filename                                                                  |
+-------------------+----------+----------+--------------+------------+--------------------------------------------------------------------------+
|2022-03-27 00:00:00|1         |12.1      |79            |3.4         |file:///C:/PalProjects/colibri-challange/code/data/landed/data_group_1.csv|
|2022-03-27 02:00:00|1         |11.6      |340           |4.2         |file:///C:/PalProjects/colibri-challange/code/data/landed/data_group_1.csv|
|2022-03-27 02:00:00|1         |10.9      |175           |1.9         |file:///C:/PalProjects/colibri-challange/code/data/landed/data_group_1.csv|
|2022-03-27 03:00:00|1         |11.4      |130           |2.1         |file:///C:/PalProjects/colibri-challange/code/d